# Polar Coding (Patched Template)
Restart kernel before running.

In [2]:
# Full Polar Coding implementation (single-cell)
# Restart kernel before running this cell to clear stale state.

import numpy as np
import matplotlib.pyplot as plt
from math import log2

# ------------------------------
# Utilities
# ------------------------------
def is_power_of_two(n): 
    return (n & (n-1)) == 0 and n > 0

def bit_reverse_indices(N):
    n = int(np.log2(N))
    br = np.zeros(N, dtype=int)
    for i in range(N):
        br[i] = int(format(i, 'b').zfill(n)[::-1], 2)
    return br

# ------------------------------
# Polar transform and matrix G
# ------------------------------
def polar_transform(u):
    """Recursive natural-order polar transform: x = u * F^{⊗n} in natural order."""
    N = len(u)
    if N == 1:
        return u.copy()
    u_even = u[0:N:2]
    u_odd  = u[1:N:2]
    upper = polar_transform((u_even ^ u_odd) % 2)
    lower = polar_transform(u_odd)
    return np.concatenate([upper, lower]).astype(np.int8)

def construct_G_nat(N):
    """Return F^{⊗n} (no bit-reversal) as integer 0/1 matrix (N x N)."""
    assert is_power_of_two(N)
    F = np.array([[1,0],[1,1]], dtype=int)
    n = int(np.log2(N))
    G = np.array([[1]], dtype=int)
    for _ in range(n):
        G = np.kron(G, F)
    return G % 2

def encode_with_G(u):
    G = construct_G_nat(len(u))
    x = (u.astype(int) @ G.astype(int)) % 2
    return x.astype(np.int8)

def polar_encode(msg, info_positions, N, u_frozen=None, use_recursive=True):
    """
    Encode:
      - msg: array length K (bits)
      - info_positions: indices in natural order where msg bits go
      - N: block length
      - u_frozen: frozen values (default all zeros)
      - use_recursive: if True use polar_transform; else use matrix multiply with G_nat
    Returns x (codeword) and u (full length N input bits)
    """
    if u_frozen is None:
        u_frozen = np.zeros(N, dtype=np.int8)
    u = np.array(u_frozen, dtype=np.int8)
    u[np.sort(info_positions)] = np.array(msg, dtype=np.int8)
    if use_recursive:
        x = polar_transform(u)
    else:
        x = encode_with_G(u)
    return x.astype(np.int8), u.astype(np.int8)

# ------------------------------
# Bhattacharyya parameter and frozen selection
# ------------------------------
def bhattacharyya_parameter(p, N):
    Z = np.array([2.0 * np.sqrt(p * (1.0 - p))])
    while len(Z) < N:
        Z_next = []
        for z in Z:
            Z_next.append(2*z - z*z)  # upper
            Z_next.append(z*z)        # lower
        Z = np.array(Z_next)
    return Z

def select_frozen_logical_to_natural(N, K, p_design):
    """
    ALTERNATIVE: choose the K best logical channels by Bhattacharyya Z
    and PLACE them directly at those natural indices (no bit-reversal).
    This often matches polar_transform + SC written in some code styles.
    """
    Z = bhattacharyya_parameter(p_design, N)
    logical_best = np.argsort(Z)[:K]   # best logical indices (0..N-1)
    # Place logical indices directly in natural positions:
    info_positions_natural = np.sort(logical_best)
    frozen_mask = np.ones(N, dtype=bool)
    frozen_mask[info_positions_natural] = False
    return frozen_mask, info_positions_natural


# ------------------------------
# Channel models and LLRs
# ------------------------------
def bsc(x, p):
    flips = (np.random.rand(len(x)) < p).astype(np.int8)
    return (x ^ flips).astype(np.int8)

def bsc_llr(y, p, eps=1e-12):
    p_safe = min(max(float(p), eps), 1.0 - eps)
    alpha = np.log((1.0 - p_safe) / p_safe)
    return (1 - 2*y) * alpha

# ------------------------------
# LLR combining functions
# ------------------------------
def f_min_sum(a, b):
    a = np.array(a); b = np.array(b)
    return np.sign(a * b) * np.minimum(np.abs(a), np.abs(b))

def f_exact(a, b, eps=1e-12):
    A = np.array(a, dtype=float)
    B = np.array(b, dtype=float)
    ta = np.tanh(A / 2.0)
    tb = np.tanh(B / 2.0)
    t = ta * tb
    t = np.clip(t, -1.0 + eps, 1.0 - eps)
    return np.log1p(t) - np.log1p(-t)  # = 2*atanh(t)

# ------------------------------
# Natural-order SC decoder (recursive)
# ------------------------------
def sc_decode(llr, frozen_mask, u_frozen=None, use_exact_f=False):
    """
    Natural-order successive cancellation decoder.
    llr: length N float array
    frozen_mask: boolean length N (True means frozen)
    u_frozen: values for frozen bits (length N)
    """
    import numpy as _np
    if u_frozen is None:
        u_frozen = _np.zeros(len(frozen_mask), dtype=_np.int8)
    llr = _np.array(llr, dtype=float)
    frozen_mask = _np.array(frozen_mask, dtype=bool)
    u_frozen = _np.array(u_frozen, dtype=_np.int8)

    def rec(llr_sub, frozen_sub, u_frozen_sub):
        n = len(llr_sub)
        if n == 1:
            if frozen_sub[0]:
                return _np.array([u_frozen_sub[0]], dtype=_np.int8)
            else:
                return _np.array([0 if llr_sub[0] >= 0 else 1], dtype=_np.int8)
        n2 = n // 2
        a = llr_sub[:n2]
        b = llr_sub[n2:]
        f = f_exact(a, b) if use_exact_f else f_min_sum(a, b)
        uhat_up = rec(f, frozen_sub[:n2], u_frozen_sub[:n2])
        sign_term = (1 - 2 * uhat_up)
        g = b + sign_term * a
        uhat_low = rec(g, frozen_sub[n2:], u_frozen_sub[n2:])
        u_up = (uhat_up ^ uhat_low).astype(_np.int8)
        return _np.concatenate([u_up, uhat_low]).astype(_np.int8)

    return rec(llr, frozen_mask, u_frozen)

# ------------------------------
# Monte-Carlo simulation driver
# ------------------------------
def monte_carlo_polar(N=128, K=None, p_design=0.05, p_channel=None,
                      trials=1000, use_exact_f=False, verbose=False, use_recursive_encoder=True):
    """
    Monte-Carlo simulation of polar code.
    Returns (ber, bler).
    """
    if not is_power_of_two(N):
        raise ValueError("N must be power of two")
    if K is None:
        K = N // 2
    if p_channel is None:
        p_channel = p_design

    frozen_mask, info_positions = select_frozen_logical_to_natural(N, K, p_design)
    u_frozen = np.zeros(N, dtype=np.int8)
    info_positions = np.sort(info_positions)
    K_actual = len(info_positions)

    bit_err = 0
    block_err = 0
    for t in range(trials):
        msg = np.random.randint(0,2,size=K_actual).astype(np.int8)
        x, u = polar_encode(msg, info_positions, N, u_frozen=u_frozen, use_recursive=use_recursive_encoder)
        y = bsc(x, p_channel)
        llr = bsc_llr(y, p_channel)
        uhat = sc_decode(llr, frozen_mask, u_frozen=u_frozen, use_exact_f=use_exact_f)
        msg_hat = uhat[info_positions]
        this_err = int(np.sum(msg_hat != msg))
        bit_err += this_err
        block_err += int(this_err > 0)
        if verbose and (t % max(1, trials//10) == 0):
            print(f"Trial {t}/{trials}: bit_err={bit_err}, block_err={block_err}")
    ber = bit_err / (trials * K_actual)
    bler = block_err / trials
    return ber, bler

# ------------------------------
# Instrumentation & sanity helpers
# ------------------------------
def instrumented_n4(p_design=0.05, p_channel=0.0, use_exact_f=True):
    """
    Deterministic instrumented check for N=4. Prints decode tree and checks match.
    """
    N = 4; K = N//2
    frozen_mask, info_positions = select_frozen_logical_to_natural(N, K, p_design)
    u_frozen = np.zeros(N, dtype=np.int8)
    msg = np.array([0,1], dtype=np.int8)
    x, u = polar_encode(msg, info_positions, N, u_frozen=u_frozen, use_recursive=True)
    print("info_positions (natural):", info_positions)
    print("u (natural):", u.tolist())
    print("x:", x.tolist())
    # noiseless
    y = x.copy()
    llr = bsc_llr(y, 1e-12)
    print("llr:", llr.tolist())
    def sc_print(llr_sub, frozen_sub, u_frozen_sub, depth=0, path="root"):
        indent = "  "*depth
        n = len(llr_sub)
        print(f"{indent}{path}: n={n}, llr_sub={llr_sub.tolist()}, frozen_sub={frozen_sub.tolist()}")
        if n==1:
            if frozen_sub[0]:
                print(f"{indent} => leaf frozen -> {u_frozen_sub[0]}")
                return np.array([u_frozen_sub[0]], dtype=np.int8)
            dec = 0 if llr_sub[0] >= 0 else 1
            print(f"{indent} => leaf decide -> {dec}")
            return np.array([dec], dtype=np.int8)
        n2 = n//2
        a = llr_sub[:n2]; b = llr_sub[n2:]
        f_llr = f_exact(a,b) if use_exact_f else f_min_sum(a,b)
        print(f"{indent} f: a={a.tolist()}, b={b.tolist()} -> f={f_llr.tolist()}")
        uhat_up = sc_print(f_llr, frozen_sub[:n2], u_frozen_sub[:n2], depth+1, path=path+"-L")
        print(f"{indent} uhat_up returned: {uhat_up.tolist()}")
        sign_term = (1 - 2 * uhat_up)
        g_llr = b + sign_term * a
        print(f"{indent} g: sign_term={sign_term.tolist()}, g_llr={g_llr.tolist()}")
        uhat_low = sc_print(g_llr, frozen_sub[n2:], u_frozen_sub[n2:], depth+1, path=path+"-R")
        print(f"{indent} uhat_low returned: {uhat_low.tolist()}")
        combined = np.concatenate([(uhat_up ^ uhat_low).astype(np.int8), uhat_low]).astype(np.int8)
        print(f"{indent} combined: {combined.tolist()}")
        return combined
    uhat = sc_print(llr, frozen_mask, u_frozen)
    print("decoded uhat:", uhat.tolist())
    print("expected u:", u.tolist())
    print("match?", np.array_equal(uhat, u))
    return uhat, u, frozen_mask, info_positions

# ------------------------------
# Plotting helper for Monte-Carlo sweep over channel p
# ------------------------------
def run_and_plot_sweep(N=128, K=None, p_design=0.05, p_list=None, trials=200, use_exact_f=False):
    if K is None:
        K = N//2
    if p_list is None:
        p_list = np.linspace(0.01, 0.2, 10)
    bers = []
    blers = []
    for p in p_list:
        print(f"Running p={p:.3f} ...", end=" ")
        ber, bler = monte_carlo_polar(N=N, K=K, p_design=p_design, p_channel=p,
                                      trials=trials, use_exact_f=use_exact_f)
        bers.append(ber)
        blers.append(bler)
        print(f"BER={ber:.4e}, BLER={bler:.4e}")
    plt.figure(figsize=(8,4))
    plt.semilogy(p_list, bers, '-o', label='BER')
    plt.semilogy(p_list, blers, '-s', label='BLER')
    plt.xlabel('BSC crossover p')
    plt.ylabel('Error rate (log scale)')
    plt.title(f'Polar code N={N}, K={K}, design p={p_design}')
    plt.grid(True, which='both', ls='--', alpha=0.6)
    plt.legend()
    plt.show()
    return p_list, np.array(bers), np.array(blers)

# ------------------------------
# Quick sanity tests (run after kernel restart)
# ------------------------------
if __name__ == "__main__":
    print("Running quick sanity checks (restart kernel first!)\n")
    # Do a deterministic instrumented check first
    print("Instrumented N=4 deterministic test:")
    uhat, u_expected, frozen_mask_example, info_pos = instrumented_n4()
    if not np.array_equal(uhat, u_expected):
        print("\nERROR: deterministic instrumented test failed. Please paste the printed output here for inspection.")
    else:
        print("\nDeterministic test passed. Now running small Monte-Carlo sanity checks...")
        for N in (4,8,16,32):
            K = N//2
            ber0, bler0 = monte_carlo_polar(N=N, K=K, p_design=0.01, p_channel=0.0, trials=200, use_exact_f=True)
            print(f"N={N} no-noise -> BER={ber0:.6f}, BLER={bler0:.6f}")
            ber5, bler5 = monte_carlo_polar(N=N, K=K, p_design=0.5, p_channel=0.5, trials=200, use_exact_f=True)
            print(f"N={N} p=0.5 -> BER={ber5:.6f}, BLER={bler5:.6f}")
    print("\nIf deterministic test failed, copy the entire printed instrumented output here and I will fix the mapping immediately.")


Running quick sanity checks (restart kernel first!)

Instrumented N=4 deterministic test:
info_positions (natural): [2 3]
u (natural): [0, 0, 0, 1]
x: [1, 1, 1, 1]
llr: [-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755]
root: n=4, llr_sub=[-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755], frozen_sub=[True, True, False, False]
 f: a=[-27.63102111592755, -27.63102111592755], b=[-27.63102111592755, -27.63102111592755] -> f=[26.937840546492907, 26.937840546492907]
  root-L: n=2, llr_sub=[26.937840546492907, 26.937840546492907], frozen_sub=[True, True]
   f: a=[26.937840546492907], b=[26.937840546492907] -> f=[26.244693365930964]
    root-L-L: n=1, llr_sub=[26.244693365930964], frozen_sub=[True]
     => leaf frozen -> 0
   uhat_up returned: [0]
   g: sign_term=[1], g_llr=[53.875681092985815]
    root-L-R: n=1, llr_sub=[53.875681092985815], frozen_sub=[True]
     => leaf frozen -> 0
   uhat_low returned: [0]
   combined: [0, 0]
 uh

In [3]:
# Full corrected Polar code: encoder replaced by exact Arikan matrix (F^{⊗n})
# Restart kernel first. Then paste and run this single cell.

import numpy as np
import matplotlib.pyplot as plt
from math import log2

# -------------------------
# Utilities
# -------------------------
def is_power_of_two(n):
    return (n & (n-1)) == 0 and n > 0

def bit_reverse_indices(N):
    n = int(np.log2(N))
    br = np.zeros(N, dtype=int)
    for i in range(N):
        br[i] = int(format(i, 'b').zfill(n)[::-1], 2)
    return br

# -------------------------
# Construct Arikan matrix G_nat = F^{⊗n} (natural order)
# -------------------------
def construct_G_nat(N):
    """Return F^{\otimes n} as an (N x N) 0/1 integer matrix."""
    assert is_power_of_two(N), "N must be power of two"
    F = np.array([[1,0],[1,1]], dtype=int)
    n = int(np.log2(N))
    G = np.array([[1]], dtype=int)
    for _ in range(n):
        G = np.kron(G, F)
    return G % 2

def encode_with_G_nat(u):
    """Encode u (0/1 vector length N) with G_nat -> x = u @ G_nat (mod 2)."""
    N = len(u)
    G = construct_G_nat(N)
    x = (u.astype(int) @ G.astype(int)) % 2
    return x.astype(np.int8)

def polar_encode_with_G(msg, info_positions, N, u_frozen=None):
    """Place msg into u at info_positions (natural), then encode using G_nat matrix."""
    if u_frozen is None:
        u_frozen = np.zeros(N, dtype=np.int8)
    u = np.array(u_frozen, dtype=np.int8)
    u[np.sort(info_positions)] = np.array(msg, dtype=np.int8)
    x = encode_with_G_nat(u)
    return x.astype(np.int8), u.astype(np.int8)

# -------------------------
# Bhattacharyya & frozen selection
# -------------------------
def bhattacharyya_parameter(p, N):
    Z = np.array([2.0 * np.sqrt(p * (1.0 - p))])
    while len(Z) < N:
        Z_next = []
        for z in Z:
            Z_next.append(2*z - z*z)  # upper
            Z_next.append(z*z)        # lower
        Z = np.array(Z_next)
    return Z

def select_frozen_direct(N, K, p_design):
    """
    Select K best logical channels by Bhattacharyya Z and place them directly
    at those natural indices (no bit-reversal). This matches encoding with G_nat.
    Returns (frozen_mask, info_positions_natural)
    """
    Z = bhattacharyya_parameter(p_design, N)
    logical_best = np.argsort(Z)[:K]
    info_positions_natural = np.sort(logical_best)
    frozen_mask = np.ones(N, dtype=bool)
    frozen_mask[info_positions_natural] = False
    return frozen_mask, info_positions_natural

# -------------------------
# Channel models & LLR
# -------------------------
def bsc(x, p):
    flips = (np.random.rand(len(x)) < p).astype(np.int8)
    return (x ^ flips).astype(np.int8)

def bsc_llr(y, p, eps=1e-12):
    p_safe = min(max(float(p), eps), 1.0 - eps)
    alpha = np.log((1.0 - p_safe) / p_safe)
    return (1 - 2*y) * alpha

# -------------------------
# LLR combines
# -------------------------
def f_min_sum(a, b):
    a = np.array(a); b = np.array(b)
    return np.sign(a * b) * np.minimum(np.abs(a), np.abs(b))

def f_exact(a, b, eps=1e-12):
    A = np.array(a, dtype=float); B = np.array(b, dtype=float)
    ta = np.tanh(A / 2.0); tb = np.tanh(B / 2.0)
    t = ta * tb
    t = np.clip(t, -1.0 + eps, 1.0 - eps)
    return np.log1p(t) - np.log1p(-t)

# -------------------------
# Natural-order SC decoder (works for G_nat)
# -------------------------
def sc_decode(llr, frozen_mask, u_frozen=None, use_exact_f=False):
    import numpy as _np
    if u_frozen is None:
        u_frozen = _np.zeros(len(frozen_mask), dtype=_np.int8)
    llr = _np.array(llr, dtype=float)
    frozen_mask = _np.array(frozen_mask, dtype=bool)
    u_frozen = _np.array(u_frozen, dtype=_np.int8)

    def rec(llr_sub, frozen_sub, u_frozen_sub):
        n = len(llr_sub)
        if n == 1:
            if frozen_sub[0]:
                return _np.array([u_frozen_sub[0]], dtype=_np.int8)
            else:
                return _np.array([0 if llr_sub[0] >= 0 else 1], dtype=_np.int8)
        n2 = n // 2
        a = llr_sub[:n2]; b = llr_sub[n2:]
        f = f_exact(a,b) if use_exact_f else f_min_sum(a,b)
        uhat_up = rec(f, frozen_sub[:n2], u_frozen_sub[:n2])
        sign_term = (1 - 2 * uhat_up)
        g = b + sign_term * a
        uhat_low = rec(g, frozen_sub[n2:], u_frozen_sub[n2:])
        u_upper = uhat_up.astype(_np.int8)
        u_lower = (uhat_up ^ uhat_low).astype(_np.int8)
        return _np.concatenate([u_upper, u_lower]).astype(_np.int8)

    return rec(llr, frozen_mask, u_frozen)

# -------------------------
# Monte-Carlo driver (consistent with G_nat)
# -------------------------
def monte_carlo_polar_matrix(N=128, K=None, p_design=0.05, p_channel=None,
                      trials=1000, use_exact_f=False, verbose=False, use_matrix_encoder=True):
    """
    Monte-Carlo for polar codes using matrix encoder G_nat and SC decoder for F^{⊗n}.
    """
    if not is_power_of_two(N):
        raise ValueError("N must be power of two")
    if K is None:
        K = N // 2
    if p_channel is None:
        p_channel = p_design

    frozen_mask, info_positions = select_frozen_direct(N, K, p_design)
    u_frozen = np.zeros(N, dtype=np.int8)
    info_positions = np.sort(info_positions)
    K_actual = len(info_positions)

    bit_err = 0
    block_err = 0
    for t in range(trials):
        msg = np.random.randint(0,2,size=K_actual).astype(np.int8)
        # encode using matrix G_nat (exact)
        x, u = polar_encode_with_G(msg, info_positions, N, u_frozen=u_frozen)
        y = bsc(x, p_channel)
        llr = bsc_llr(y, p_channel)
        # decode using SC for F^{⊗n}
        uhat = sc_decode(llr, frozen_mask, u_frozen=u_frozen, use_exact_f=use_exact_f)
        msg_hat = uhat[info_positions]
        this_err = int(np.sum(msg_hat != msg))
        bit_err += this_err
        block_err += int(this_err > 0)
        if verbose and (t % max(1, trials//10) == 0):
            print(f"Trial {t}/{trials}: bit_err={bit_err}, block_err={block_err}")
    ber = bit_err / (trials * K_actual)
    bler = block_err / trials
    return ber, bler

# -------------------------
# Instrumented N=4 diagnostic
# -------------------------
def instrumented_n4_matrix(p_design=0.05):
    N = 4; K = 2
    frozen_mask, info_positions = select_frozen_direct(N, K, p_design)
    u_frozen = np.zeros(N, dtype=np.int8)
    msg = np.array([0,1], dtype=np.int8)
    x, u = polar_encode_with_G(msg, info_positions, N, u_frozen=u_frozen)
    print("info_positions (natural):", info_positions)
    print("u (natural):", u.tolist())
    print("x:", x.tolist())
    y = x.copy()
    llr = bsc_llr(y, 1e-12)
    print("llr:", llr.tolist())
    # instrument SC recursion
    def sc_print(llr_sub, frozen_sub, u_frozen_sub, depth=0, path="root"):
        indent = "  "*depth
        n = len(llr_sub)
        print(f"{indent}{path}: n={n}, llr_sub={llr_sub.tolist()}, frozen_sub={frozen_sub.tolist()}")
        if n==1:
            if frozen_sub[0]:
                print(f"{indent} => leaf frozen -> {u_frozen_sub[0]}")
                return np.array([u_frozen_sub[0]], dtype=np.int8)
            dec = 0 if llr_sub[0] >= 0 else 1
            print(f"{indent} => leaf decide -> {dec}")
            return np.array([dec], dtype=np.int8)
        n2 = n//2
        a = llr_sub[:n2]; b = llr_sub[n2:]
        f_llr = f_exact(a,b)
        print(f"{indent} f: a={a.tolist()}, b={b.tolist()} -> f={f_llr.tolist()}")
        uhat_up = sc_print(f_llr, frozen_sub[:n2], u_frozen_sub[:n2], depth+1, path=path+"-L")
        print(f"{indent} uhat_up returned: {uhat_up.tolist()}")
        sign_term = (1 - 2 * uhat_up)
        g_llr = b + sign_term * a
        print(f"{indent} g: sign_term={sign_term.tolist()}, g_llr={g_llr.tolist()}")
        uhat_low = sc_print(g_llr, frozen_sub[n2:], u_frozen_sub[n2:], depth+1, path=path+"-R")
        print(f"{indent} uhat_low returned: {uhat_low.tolist()}")
        combined = np.concatenate([uhat_up.astype(np.int8), (uhat_up ^ uhat_low).astype(np.int8)]).astype(np.int8)
        print(f"{indent} combined: {combined.tolist()}")
        return combined
    uhat = sc_print(llr, frozen_mask, u_frozen)
    print("decoded uhat:", uhat.tolist())
    print("expected u:", u.tolist())
    print("match?", np.array_equal(uhat, u))
    return uhat, u, frozen_mask, info_positions

# -------------------------
# Quick test (restart kernel then run)
# -------------------------
if __name__ == "__main__":
    print("Running instrumented N=4 matrix-based test (restart kernel first!)\n")
    uhat, u_expected, frozen_mask_ex, info_pos = instrumented_n4_matrix()
    if np.array_equal(uhat, u_expected):
        print("\nSUCCESS: deterministic matrix encoder + SC decoder match.")
        print("Now you can run monte_carlo_polar_matrix(...) and plotting helpers.")
    else:
        print("\nFAIL: mismatch persists. Paste the printed trace here and I will fix the last detail.")


Running instrumented N=4 matrix-based test (restart kernel first!)

info_positions (natural): [2 3]
u (natural): [0, 0, 0, 1]
x: [1, 1, 1, 1]
llr: [-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755]
root: n=4, llr_sub=[-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755], frozen_sub=[True, True, False, False]
 f: a=[-27.63102111592755, -27.63102111592755], b=[-27.63102111592755, -27.63102111592755] -> f=[26.937840546492907, 26.937840546492907]
  root-L: n=2, llr_sub=[26.937840546492907, 26.937840546492907], frozen_sub=[True, True]
   f: a=[26.937840546492907], b=[26.937840546492907] -> f=[26.244693365930964]
    root-L-L: n=1, llr_sub=[26.244693365930964], frozen_sub=[True]
     => leaf frozen -> 0
   uhat_up returned: [0]
   g: sign_term=[1], g_llr=[53.875681092985815]
    root-L-R: n=1, llr_sub=[53.875681092985815], frozen_sub=[True]
     => leaf frozen -> 0
   uhat_low returned: [0]
   combined: [0, 0]
 uhat_up returned: [0, 0]

<>:25: SyntaxWarning: invalid escape sequence '\o'
<>:25: SyntaxWarning: invalid escape sequence '\o'
/tmp/ipykernel_8558/4129746908.py:25: SyntaxWarning: invalid escape sequence '\o'
  """Return F^{\otimes n} as an (N x N) 0/1 integer matrix."""
